In [1]:
import numpy as np
import pandas as pd
import keras
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from gensim.models.doc2vec import LabeledSentence
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout, Concatenate
import keras.layers.merge
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import LSTM, Bidirectional
from keras.models import Model
from keras.callbacks import EarlyStopping
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
import nltk
from nltk.tokenize import RegexpTokenizer
import re
import codecs
import matplotlib.pyplot as plt
from gensim.models import Doc2Vec
import pickle

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\User\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
EMBEDDING_DIM = 700 # how big is each word vector
MAX_VOCAB_SIZE = 175303 # how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH = 75 # max number of words in a comment to use

#training params
batch_size = 256  
num_epochs = 20 

In [3]:
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r'\d+','<number>',text)
    return text

In [4]:
clean_train_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/train.csv")
# clean_train_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/data_train_full.csv")
clean_train_comments['content'] = clean_train_comments['content'].apply(preprocess)
clean_train_comments.head()

,content,polarity
0,gerindra alihkan rekomendasi ke agus an tanri ...,neutral
1,cuci tangan pakai sunlight stelah itu pakai sa...,neutral
2,kasus toko obat digerebek fpi propam akan peri...,neutral
3,menkeu melemah nya rupiah lebih berpengaruh pa...,neutral
4,minyak jarak castor oil <number> ml,neutral


In [5]:
tokenizer = RegexpTokenizer(r'\w+')
clean_train_comments['content'] = clean_train_comments['content'].astype('str') 
clean_train_comments.dtypes
clean_train_comments['tokens'] = clean_train_comments['content'].apply(tokenizer.tokenize)
clean_train_comments['sentiment'] = clean_train_comments['polarity'].astype('category').cat.codes
   
clean_train_comments.head()

,content,polarity,tokens,sentiment
0,gerindra alihkan rekomendasi ke agus an tanri ...,neutral,"[gerindra, alihkan, rekomendasi, ke, agus, an,...",1
1,cuci tangan pakai sunlight stelah itu pakai sa...,neutral,"[cuci, tangan, pakai, sunlight, stelah, itu, p...",1
2,kasus toko obat digerebek fpi propam akan peri...,neutral,"[kasus, toko, obat, digerebek, fpi, propam, ak...",1
3,menkeu melemah nya rupiah lebih berpengaruh pa...,neutral,"[menkeu, melemah, nya, rupiah, lebih, berpenga...",1
4,minyak jarak castor oil <number> ml,neutral,"[minyak, jarak, castor, oil, number, ml]",1


In [6]:
clean_test_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/test.csv")
# clean_test_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/data_testing_full.csv")
clean_test_comments['content'] = clean_test_comments['content'].apply(preprocess)
clean_test_comments.head()

,content,polarity
0,kemarin gue datang ke tempat makan baru yang a...,negative
1,kayak nya sih gue tidak akan mau balik lagi ke...,negative
2,kalau dipikirpikir sebenarnya tidak ada yang b...,negative
3,ini pertama kalinya gua ke bank buat ngurusin ...,negative
4,waktu sampai dengan gue pernah disuruh ibu lat...,negative


In [7]:
clean_test_comments['content'] = clean_test_comments['content'].astype('str') 
clean_test_comments.dtypes
clean_test_comments["tokens"] = clean_test_comments["content"].apply(tokenizer.tokenize)
clean_test_comments['sentiment'] = clean_test_comments['polarity'].astype('category').cat.codes

clean_test_comments.head()

,content,polarity,tokens,sentiment
0,kemarin gue datang ke tempat makan baru yang a...,negative,"[kemarin, gue, datang, ke, tempat, makan, baru...",0
1,kayak nya sih gue tidak akan mau balik lagi ke...,negative,"[kayak, nya, sih, gue, tidak, akan, mau, balik...",0
2,kalau dipikirpikir sebenarnya tidak ada yang b...,negative,"[kalau, dipikirpikir, sebenarnya, tidak, ada, ...",0
3,ini pertama kalinya gua ke bank buat ngurusin ...,negative,"[ini, pertama, kalinya, gua, ke, bank, buat, n...",0
4,waktu sampai dengan gue pernah disuruh ibu lat...,negative,"[waktu, sampai, dengan, gue, pernah, disuruh, ...",0


In [8]:
all_training_words = [word for tokens in clean_train_comments["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in clean_train_comments["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

22059 words total, with a vocabulary size of 4708
Max sentence length is 73


In [9]:
all_test_words = [word for tokens in clean_test_comments["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in clean_test_comments["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

1474 words total, with a vocabulary size of 607
Max sentence length is 52


In [10]:
def labelize_text(text,label):
    result = []
    prefix = label
    for i, t in zip(text.index, text):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result
  
x_train = labelize_text(clean_train_comments["content"], 'TRAIN')
x_validation = labelize_text(clean_test_comments["content"], 'TEST')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [11]:
word2vec = Word2Vec.load('./prosa-w2v/prosa.vec')
# word2vec = Word2Vec.load('./vectorizer/prosa/word2vec.model')

In [12]:
tfidf = pickle.load(open('./vectorizer/prosa/tfidf.pickle', 'rb'))
model_dbow = Doc2Vec.load("./vectorizer/prosa/model_dbow.model")
model_dmc = Doc2Vec.load("./vectorizer/prosa/model_dmc.model")
model_dmm = Doc2Vec.load("./vectorizer/prosa/model_dmm.model")

In [13]:
def build_doc_Vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += np.append(model_dbow[word] * tfidf[word], model_dmm[word] * tfidf[word])
            count += 1
        except KeyError: 
            continue
    if count != 0:
        vec /= count
    return vec

In [14]:
def build_Vector(tokens, word_size, doc_size):
    doc_vec = build_doc_Vector(tokens, doc_size)
    vec = np.zeros((MAX_SEQUENCE_LENGTH - len(tokens), doc_size + word_size))
    for word in tokens:
        try:
            word_vec = np.append(doc_vec, word2vec[word])
            vec = np.append(vec, word_vec)
        except KeyError: 
            word_vec = np.append(doc_vec, np.zeros((1, word_size)))
            vec = np.append(vec, word_vec)
            continue
    vec.reshape(MAX_SEQUENCE_LENGTH, doc_size + word_size)
    return vec

In [15]:
train_vecs = np.concatenate([[build_Vector(z, 500, 200)] for z in tqdm(map(lambda x: x.words, x_train))])
val_vecs = np.concatenate([[build_Vector(z, 500, 200)] for z in tqdm(map(lambda x: x.words, x_validation))])

0it [00:00, ?it/s]C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
935it [00:02, 334.33it/s]
100it [00:00, 436.20it/s]


In [16]:
num_data = len(train_vecs)
num_data_val = len(val_vecs)

train_vecs = train_vecs.reshape((num_data, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM))
val_vecs = val_vecs.reshape((num_data_val, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM))

In [17]:
def ConvNet(max_sequence_length, embedding_dim, labels_index, trainable=False, extra_conv=True):

    sequence_input = Input(shape=(max_sequence_length, embedding_dim,), dtype='float32')

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    convs = []
    filter_sizes = [3,4,5]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(sequence_input)
        l_pool = MaxPooling1D(pool_size=3)(l_conv)
        convs.append(l_pool)

    #l_merge = Merge(mode='concat', concat_axis=1)(convs)
    l_merge = Concatenate(axis=1)(convs)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    conv = Conv1D(filters=128, kernel_size=3, activation='relu')(sequence_input)
    pool = MaxPooling1D(pool_size=3)(conv)

    if extra_conv==True:
        x = Dropout(0.5)(l_merge)  
    else:
        # Original Yoon Kim model
        x = Dropout(0.5)(pool)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    preds = Dense(3, activation='softmax')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc']) 
    model.summary()
    return model

In [18]:
y_tr = clean_train_comments['sentiment'].values
y_ts = clean_test_comments['sentiment'].values

In [19]:
x_train = train_vecs
y_train = y_tr

x_test = val_vecs
y_test = y_ts

In [20]:
model = ConvNet(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM, 1, False)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 75, 700)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 73, 128)      268928      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 72, 128)      358528      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 71, 128)      448128      input_1[0][0]                    
__________________________________________________________________________________________________
max_poolin

In [21]:
hist = model.fit(x_train, to_categorical(y_train), epochs=num_epochs, validation_data=(x_test, to_categorical(y_test)), batch_size=batch_size)

Train on 935 samples, validate on 100 samples
Epoch 1/20
935/935 [==============================] - ETA: 15s - loss: 2.5003 - acc: 0.30 - ETA: 7s - loss: 3.2057 - acc: 0.3555 - ETA: 2s - loss: 2.7511 - acc: 0.438 - 14s 15ms/step - loss: 2.6892 - acc: 0.4588 - val_loss: 2.3935 - val_acc: 0.3900
Epoch 2/20
935/935 [==============================] - ETA: 7s - loss: 1.6128 - acc: 0.562 - ETA: 4s - loss: 1.4729 - acc: 0.550 - ETA: 1s - loss: 1.3341 - acc: 0.558 - 11s 12ms/step - loss: 1.2719 - acc: 0.5668 - val_loss: 1.3291 - val_acc: 0.4300
Epoch 3/20
935/935 [==============================] - ETA: 7s - loss: 0.8500 - acc: 0.632 - ETA: 4s - loss: 0.8552 - acc: 0.627 - ETA: 1s - loss: 0.8482 - acc: 0.625 - 11s 12ms/step - loss: 0.8256 - acc: 0.6417 - val_loss: 1.2262 - val_acc: 0.3600
Epoch 4/20
935/935 [==============================] - ETA: 7s - loss: 0.7335 - acc: 0.734 - ETA: 4s - loss: 0.7813 - acc: 0.714 - ETA: 1s - loss: 0.7381 - acc: 0.729 - 11s 12ms/step - loss: 0.7201 - acc: 0.733

In [22]:
# model.save('./model/yoon_kim_3_pv/cnn_model_04.h5') 

In [29]:
# model = load_model('./model/yoon_kim_3_pv/cnn_model_04.h5')

y_predict = model.predict(val_vecs, batch_size=256, verbose=1)
y_predict = np.argmax(y_predict, axis=1)

100/100 [==============================] - 1s 5ms/step
             precision    recall  f1-score   support

          0  0.48611111 0.87500000 0.62500000        40
          1  0.47368421 0.45000000 0.46153846        20
          2  0.77777778 0.17500000 0.28571429        40

avg / total  0.60029240 0.51000000 0.45659341       100



In [31]:
print(classification_report(y_test, y_predict, labels = [0, 1, 2], digits=8))

             precision    recall  f1-score   support

          0  0.48611111 0.87500000 0.62500000        40
          1  0.47368421 0.45000000 0.46153846        20
          2  0.77777778 0.17500000 0.28571429        40

avg / total  0.60029240 0.51000000 0.45659341       100

